In [ ]:
import os
import math
import nglview
from ipywidgets import widgets

from moleidoscope.linker import Linker, view_linkers
from moleidoscope.geo.coor import Coor
from moleidoscope.mirror import Mirror
from moleidoscope.line import Line

## Display Linkers ##

### 1. Displaying a single linker ###

In [ ]:
Linker(21).view()                   # Create new linker and view

### 2. Displaying multiple linkers ###

In [ ]:
link1 = Linker(14)                  # Create Linker 1
link2 = Linker(1254)                  # Create Linker 2
link2.center(coor=[-8, 2, 0])      # Center Linker 2 so they do not overlap
link3 = Linker(123)                  # Create Linker 3
link3.center(coor=[7, 2, 0])      # Center Linker 2 so they do not overlap
view_linkers(link1, link2, link3)         # View multiple linkers

### 3. Displaying selected linker from library ###

In [ ]:
view = nglview.NGLWidget()                                     # Initialize nglview window

def set_camera(orthographic):                                  # Select camera type using widgets
    if orthographic:
        view.camera = 'orthographic'                                  
    else:
        view.camera = 'perspective'                           

def show_linker(idx):
    components = view._ngl_component_ids                       # List all components in view
    if len(components) > 0:                                    # Remove all components
        for i in components:                                   # .
            view.remove_component(i)                           # .
    link = Linker(int(idx))                                    # Create linker with selected index
    link.add_to_view(view)                                     # Add linker to scene
    print('Displaying linker: %s (%i atoms)' % (link.name, link.num_of_atoms))

widgets.interact(show_linker, idx=(0, 8265))                   # Get linker index from slider
widgets.interact(set_camera, orthographic=True)
view                                                           # View structures

## Translation ##

In [ ]:
view = nglview.NGLWidget()                                    # Initialize nglview window
link = Linker(33)
s = nglview.FileStructure(link.export())                      # Create structure using pdb path
view.add_component(s)                                         # Add structure to view

def translate_linker(x):
    components = view._ngl_component_ids                      # List all components in view
    if len(components) > 1:                                   # Remove all components
        for i in components:                                  # .
            view.remove_component(i)                          # Translate linker 
    link.translate([-x, 0, 0])                                # Export linker and return path
    s = nglview.FileStructure(link.export(file_name='view'))  # Create structure using pdb path
    view.add_component(s)
    print('Translating linker in x direction by %i Angstroms' % x)
    
widgets.interact(translate_linker, x=(-20., 20., 1))          # Get linker index from slider
view                                                          # View structures

## Reflection in common planes ##

In [ ]:
from IPython.display import display
button1 = widgets.Button(description='xy plane')
button2 = widgets.Button(description='xz plane')
button3 = widgets.Button(description='yz plane')
display(button1, button2, button3)

view = nglview.NGLWidget()                                    # Initialize nglview window


link = Linker(33)                                             # Create a new linker from library
link.center(coor=[5, 5, 5])                                   # Center the linker to [5, 5, 5]

link2 = link.reflect('xy', translate=None)                    # Reflect along XY plane
link3 = link.reflect('xz', translate=None)                    # Reflect along XZ plane
link4 = link.reflect('yz', translate=None)                    # Reflect along YZ plane

link.add_to_view(view)
Line.xyz(size=20).add_to_view(view)                                        # xyz axis
Mirror('xy', size=10).to_linker(atom_type='N').add_to_view(view)           # XY plane -> blue
Mirror('yz', size=10).to_linker(atom_type='O').add_to_view(view)           # YZ plane -> red
Mirror('xz', size=10).to_linker(atom_type='C').add_to_view(view)           # XZ plane -> gray

def reflect_xy(plane):
    link2 = link.reflect('xy')
    link2.add_to_view(view)

def reflect_xz(plane):
    link2 = link.reflect('xz')
    link2.add_to_view(view)
    
def reflect_yz(plane):
    link2 = link.reflect('yz')
    link2.add_to_view(view)

button1.on_click(reflect_xy)
button2.on_click(reflect_xz)
button3.on_click(reflect_yz)
view

## Reflection in arbitrary planes

In [ ]:
link = Linker(142)

# Define a plane in 3D space for reflection
p1 = [1, 10, 6]
p2 = [10, 4, -9]
p3 = [-0.1, 4, 5]
mir = Mirror(p1, p2, p3, size=0.9)                                # Create mirror using 3 points

link.center(mirror=[mir, 0.06])                                 # Center the linker according to mirror

link2 = link.reflect(mir)
view_linkers(link, link2, mir.to_linker(atom_type='N'))

## Rotation

In [ ]:
# ROTATION -----------------------------------------------------------------------------------
view = nglview.NGLWidget()                                  # Initialize nglview window

y_axis = Line([0, -7, 0], [0, 6, 0]).to_linker()            # Create y axis line
y_axis.add_to_view(view)                                    # Add y axis line to scene

link = Linker(34)                                           # Create new linker
link.center(coor=[5,0,0])                                   # Center linker to [5, 0, 0]
link.add_to_view(view)                                      # Add linker to scene

def rotate_linker(d):                                       # Rotate linker according to slider value
    components = view._ngl_component_ids                    # List all components in view
    if len(components) > 2:                                 # Remove if more then two objects in scene
        view.remove_component(components[-1])               # .
    deg = math.radians(d)                                   # Convert degree to radians
    rotation_info = [deg, [0, 0, 0], [0, 1, 0]]             # Rotation info: [radians, p1, p2] 
    link2 = link.rotate(rotation_info)                      # Rotate linker around the line <p1-----p2>
    link2.add_to_view(view)                                 # Add linker to scene
    print('Rotating %s around y axis for %i degrees' % (link2.name, d))

widgets.interact(rotate_linker, d=(0, 360, 90))             # Get rotation degrees from slider
view

## Rotoreflection (Improper Rotation)

In [ ]:
from IPython.display import display
button1 = widgets.Button(description='Rotate around y axis')
button2 = widgets.Button(description='Reflect xz plane')
display(button1, button2)

# ROTOREFLECTION (Improper Rotation) ---------------------------------------------
view = nglview.NGLWidget()                                  # Initialize nglview window

z_axis = Line([0, -7, 0], [0, 6, 0]).to_linker()            # Create z axis line
z_axis.add_to_view(view)                                    # Add z axis line to scene

link = Linker(17)
link.center(coor=[5, 0, 5])
link.add_to_view(view)

mir = Mirror('xz')
plane = mir.to_linker()
plane.translate([-5, -0, -5])
plane.add_to_view(view)
link2 = link.rotate(rotation_info)

def rotate(d):
    rotation_info = [math.pi, [0, 0, 0], [0, 1, 0]]
    link2 = link.rotate(rotation_info)
    link2.add_to_view(view)
    
def reflect(p):
    components = view._ngl_component_ids
    #view.remove_component(components[-1])
    link3 = link2.reflect('xz')
    link3.add_to_view(view)

button1.on_click(rotate)
button2.on_click(reflect)
view

In [ ]:
# Additional Methods

In [ ]:
# ROTOREFLECTION (Improper Rotation) ---------------------------------------------
view = nglview.NGLWidget()                                  # Initialize nglview window

z_axis = Line([0, -7, 0], [0, 6, 0]).to_linker()            # Create z axis line
z_axis.add_to_view(view)                                    # Add z axis line to scene

link = Linker(17)
link.center(coor=[5, 0, 5])
link.add_to_view(view)

# Determine rotation -> [angle, p1, p2] where p1----p2 is the rotation axis
rotation_info = [math.pi, [0, 0, 0], [0, 1, 0]]
# Determine reflection plane as 'xy

mir = Mirror('xz')
plane = mir.to_linker()
plane.translate([0, 5, 0])
plane.add_to_view(view)
link2 = link.rotoreflect(rotation_info, mir, translate=0)
link2.add_to_view(view)

view

In [ ]:
# Adding linker to scene using NGLWidget
view = nglview.NGLWidget()
linker_path = Linker(12).export()
s = nglview.FileStructure(linker_path)
view.add_component(s)
view

In [ ]:
# Refltecion in common planes
link = Linker(33)                                             # Create a new linker from library
link.center(coor=[5, 5, 5])                                   # Center the linker to [5, 5, 5]

link2 = link.reflect('xy', translate=None)                    # Reflect along XY plane
link3 = link.reflect('xz', translate=None)                    # Reflect along XZ plane
link4 = link.reflect('yz', translate=None)                    # Reflect along YZ plane

view_linkers(link, link2, link3, link4,
             Line.xyz(size=20),                                # xyz axis
             Mirror('xy', size=10).to_linker(atom_type='N'),   # XY plane -> blue
             Mirror('yz', size=10).to_linker(atom_type='O'),   # YZ plane -> red
             Mirror('xz', size=10).to_linker(atom_type='C'))   # XZ plane -> gray